In [ ]:
import pandas as pd
df=pd.read_csv('C:\\Users\\HP\\Desktop\\fake news\\Train.csv')
df.head()
df=df.dropna()
df['Text_final']=df['Text']+df['Text_Tag']
df.drop(['Text','Text_Tag'],axis=1,inplace=True)
x=df.drop('Labels',axis=1)
y=df['Labels']
print(x.shape)
print(y.shape)
print(x)

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
# set vocabulary size
voc_size=5000
messages=x.copy()
messages.reset_index(inplace=True)
print(messages)

       index                                         Text_final
0          0  Says the Annies List political group supports ...
1          1  When did the decline of coal start? It started...
2          2  Hillary Clinton agrees with John McCain "by vo...
3          3  Health care reform legislation is likely to ma...
4          4  The economic turnaround started at the end of ...
...      ...                                                ...
10233  10235  There are a larger number of shark attacks in ...
10234  10236  Democrats have now become the party of the [At...
10235  10237  Says an alternative to Social Security that op...
10236  10238  On lifting the U.S. Cuban embargo and allowing...
10237  10239  The Department of Veterans Affairs has a manua...

[10238 rows x 2 columns]


In [3]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    #print(i)
    review=re.sub('[^a-zA-Z]',' ',messages['Text_final'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2297, 1653, 1792, 2655, 983, 4848, 1111, 512, 4046, 1635, 4046],
 [3559,
  4378,
  511,
  511,
  129,
  2313,
  3945,
  511,
  3798,
  2570,
  3416,
  2598,
  1389,
  2859,
  3036,
  4520,
  2729,
  1096],
 [4705,
  1316,
  4292,
  1260,
  2272,
  3212,
  365,
  3416,
  1389,
  3702,
  3100,
  3382,
  3278,
  1820],
 [1375, 4423, 294, 3535, 2038, 2013, 2538, 1652, 1729, 23, 1375, 4423],
 [3679, 1014, 511, 199, 3693, 3374, 2729],
 [3851,
  1139,
  511,
  1765,
  3431,
  1993,
  902,
  4722,
  3264,
  3614,
  4592,
  2600,
  3431,
  2400,
  3333,
  1814],
 [2526, 20, 2491, 4158, 1159, 1993, 4523, 2080],
 [4207,
  4352,
  3278,
  4789,
  3431,
  1993,
  4005,
  416,
  2441,
  4789,
  3856,
  2811,
  294,
  820,
  4785,
  2811],
 [4164,
  3945,
  3267,
  147,
  4946,
  3728,
  3438,
  2287,
  3052,
  3649,
  340,
  16,
  1619,
  1631,
  3686,
  2843,
  2729],
 [2297,
  1447,
  2650,
  2107,
  3042,
  4497,
  4858,
  3789,
  4988,
  959,
  3212,
  3770,
  3267,
  4138,
  2767,
  3770,
  3

In [5]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)



[[   0    0    0 ... 4046 1635 4046]
 [   0    0 3559 ... 4520 2729 1096]
 [   0    0    0 ... 3382 3278 1820]
 ...
 [   0 2297 2445 ... 4101 2905 4287]
 [   0    0    0 ...  433 3278 1820]
 [   0    0 2386 ... 1375 4423  206]]


In [6]:
#create model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [18]:

import numpy as np
x_final=np.array(embedded_docs)
y_final=np.array(y)

from sklearn.model_selection import train_test_split
x_train,y_train,x_test,y_test=train_test_split(x_final,y_final,test_size=.33,random_state=42)
#y_train = pd.read_csv(('C:\\Users\\HP\\Desktop\\fake news\\Train.csv'), index_col=[0])
print(x_final.shape)
print(y_final.shape)
print(x_test.shape)
print(y_test.shape)
#x_test=np.reshape(x_test,(3379,))
model.fit(x_train,y_train,epochs=10,batch_size=64)

(10238, 20)
(10238,)
(6859,)
(3379,)


ValueError: Data cardinality is ambiguous:
  x sizes: 6859
  y sizes: 3379
Make sure all arrays contain the same number of samples.

In [8]:
#model training
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import tensorflow as tf
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Flatten

from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
sequences = [
    [1, 2, 3, 4],
       [1, 2, 3],
             [1]
    ]
tf.keras.utils.pad_sequences(
    sequences,
    maxlen=None,
    dtype='float',
    padding='pre',
    truncating='pre',
    value=0.0
)


padded = pad_sequences(sequences)
x_train = np.expand_dims(padded, axis = 0)
y_train = np.array([1,0,1])
y_train = y_train.reshape(1,-1)

model = Sequential([tf.keras.Input(shape=(1,)), tf.keras.layers.Dense(1)])

model.add(Dropout(0.5))
model.add(Dense(3, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='relu'))
model.compile (
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001))
x_train = tf.constant([12.2, 12.1]).reshape(-1,1)

# Create labels (using tensors)
y_train = tf.constant([12.5, 12.3]).reshape(-1,1)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
predict_x=model.predict(x_test) 
classes_x=np.argmax(predict_x,axis=1)

print(classes_x)


215/215 [==============================] - 1s 2ms/step
[4 4 4 ... 4 4 4]


In [9]:
y_pred=model.predict(classes_x)
df1=pd.DataFrame(y_pred)
y_pred=df1.iloc[0:3379,1]
print(y_pred.shape)
print(y_test.shape)

215/215 [==============================] - 1s 2ms/step
(3379,)
(3379,)


In [10]:
print(y_pred.shape)
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred.round())

(3379,)


0.16484166913287954